In [1]:
import pandas as pd
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [12]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [2]:
independent=dataset.drop(['Purchased'],axis=1)
dependent=dataset['Purchased']

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
Preprocessor=ColumnTransformer(transformers=[
    ('pre',OneHotEncoder(drop='first'),['Gender'])
], 
     remainder='passthrough',
                              force_int_remainder_cols=False)
#force_int_remainder_cols=False -->this is optional and since we were getting some warning, we are using this

In [18]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(independent,dependent,test_size=0.3,random_state=0)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
pipe=Pipeline(steps=[
    ('preprocess',Preprocessor),
    ('model',DecisionTreeClassifier(random_state=0))
])

In [20]:
from sklearn.model_selection import GridSearchCV
#criterion{“gini”, “entropy”, “log_loss”}, default=”gini”
#splitter{“best”, “random”}, default=”best”
#max_featuresint, float or {“sqrt”, “log2”}, default=None
param_grid={"model__criterion":['entropy','log_loss','gini'],"model__splitter":['best','random'],"model__max_features":['sqrt','log2']}
grid=GridSearchCV(pipe,param_grid,n_jobs=-1,verbose=3)
grid.fit(Xtrain,Ytrain)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(force_int_remainder_cols=False,
                                                          remainder='passthrough',
                                                          transformers=[('pre',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Gender'])])),
                                       ('model',
                                        DecisionTreeClassifier(random_state=0))]),
             n_jobs=-1,
             param_grid={'model__criterion': ['entropy', 'log_loss', 'gini'],
                         'model__max_features': ['sqrt', 'log2'],
                         'model__splitter': ['best', 'random']},
             verbose=3)

In [22]:
print(grid.best_params_)

{'model__criterion': 'gini', 'model__max_features': 'sqrt', 'model__splitter': 'best'}


In [21]:
Ypred=grid.predict(Xtest)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Ytest,Ypred)
print(cm)

[[70  9]
 [ 5 36]]


In [23]:
from sklearn.metrics import classification_report
clf_report=classification_report(Ytest,Ypred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.93      0.89      0.91        79
           1       0.80      0.88      0.84        41

    accuracy                           0.88       120
   macro avg       0.87      0.88      0.87       120
weighted avg       0.89      0.88      0.88       120

